In [1]:
import os
import glob
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches

from tqdm import tqdm

from PIL import Image, ImageOps

import sys

effdet_path = "../third/effdet"
sys.path.append(effdet_path)
timm_path = "../third/timm-pytorch-image-models"
sys.path.append(timm_path)
import timm
from timm.data import IMAGENET_DEFAULT_MEAN, IMAGENET_DEFAULT_STD
from matplotlib import patches
import sklearn
import os
from tqdm import tqdm

omega_path = "../third/omegaconf"
sys.path.append(omega_path)

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torchvision
from torchvision.io import read_image
import torchvision.transforms as T

import torchvision.transforms.functional as TF
import torchvision.models as models

import albumentations as A
from albumentations.pytorch import ToTensorV2

device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [56]:
from effdet import get_efficientdet_config, EfficientDet, DetBenchTrain
from effdet.efficientdet import HeadNet

class EfficientSeg(EfficientDet):
    def forward(self, x):
        x = self.backbone(x)
        print(len(x))
        for xi in x:
            print(xi.shape)
        # print(x)
        x = self.fpn(x)

        for xi in x:
            print(xi.shape)
        return x

def get_model():
    config = get_efficientdet_config('tf_efficientdet_lite0')
    config["image_size"] = (256, 256)
    config["min_level"] = 1
    config["max_level"] = 5
    config["backbone_indices"] = (0, 1, 2, 3, 4)
    model = EfficientSeg(config, pretrained_backbone=True)

    return model

model = get_model().to(device)
# model
input = torch.randn(2, 3, 256, 256).to(device)
y = model(input)

5
torch.Size([2, 16, 128, 128])
torch.Size([2, 24, 64, 64])
torch.Size([2, 40, 32, 32])
torch.Size([2, 112, 16, 16])
torch.Size([2, 320, 8, 8])
torch.Size([2, 64, 128, 128])
torch.Size([2, 64, 64, 64])
torch.Size([2, 64, 32, 32])
torch.Size([2, 64, 16, 16])
torch.Size([2, 64, 8, 8])


In [28]:

# print(y)
for iy in y:
    print(len(iy))
    print(iy[0].shape)
    print(iy[-1].shape)

5
torch.Size([2, 810, 32, 32])
torch.Size([2, 810, 2, 2])
5
torch.Size([2, 36, 32, 32])
torch.Size([2, 36, 2, 2])


In [ ]:
class SegDataset(Dataset):
    def __init__(self, df, image_dir, seg_dir, transform=None):
        self.df = df
        self.image_dir = image_dir
        self.seg_dir = seg_dir
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        df = self.df.iloc[idx]
        UID = df.name

        index = int(df[f"_index"])

        slice_img = Image.open(os.path.join(self.image_dir, UID, f"{index}.jpeg"))
        label_img = Image.open(os.path.join(self.seg_dir, UID, f"{index}.png"))

        left, top, right, bottom = df[['left','top','right','bottom']]
        slice_img = TF.crop(slice_img, top, left, bottom-top, right-left)
        label_img = TF.crop(label_img, top, left, bottom-top, right-left)

        # label_img = np.round(np.asarray(label_img, np.uint8) // 32)
        if self.transform:
            slice_img, label_img = self.transform(slice_img, label_img)

        return slice_img, label_img